<a href="https://colab.research.google.com/github/davanstrien/people-iiif/blob/main/00_wikidata_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install httpx loguru fastcore sparqlwrapper tqdm

     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 61kB 4.2MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 235kB 48.5MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 


```sql
# ?theme ?themeLabel ?tgm ?item ?itemLabel ?url
select ?themeLabel ?tgm ?url {
  # Get themes that have LTGCM subject headings
  ?theme wdt:P5160 ?tgm .
  # Get the items that dipict the theme
  ?item wdt:P180 ?theme .
  # limit to Photographs
  ?item wdt:P31 wd:Q125191 .
  # Image url P6108
  ?item wdt:P18 ?url
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
   }
}
```

In [2]:
import pandas as pd
import httpx
import io
from pathlib import Path
from fastcore.all import *
from PIL import Image
import hashlib
from loguru import logger

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
path = Path('/content/drive/MyDrive/IIIF_AI_project')

In [5]:
endpoint_url = "https://query.wikidata.org/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """# ?theme ?themeLabel ?tgm ?item ?itemLabel ?url
select ?themeLabel ?tgm ?url where {
  # Get themes that have LTGCM subject headings
  ?theme wdt:P5160 ?tgm .
  # Get the items that dipict the theme
  ?item wdt:P180 ?theme .
  # limit to Photographs
  ?item wdt:P31 wd:Q125191 .
  # Image url P6108
  ?item wdt:P18 ?url
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
   }
}  order by desc(?count) """


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)


In [51]:
df = pd.DataFrame.from_dict(results["results"]["bindings"])

In [52]:
df['tgm'] = df['tgm'].str['value']

In [53]:
df['themeLabel'] = df['themeLabel'].str['value']

In [54]:
df['url' ] = df['url'].str['value']

In [55]:
df['urlhash'] = pd.util.hash_pandas_object(df.url, index=False)

In [56]:
df

,tgm,themeLabel,url,urlhash
0,tgm009985,sport,http://commons.wikimedia.org/wiki/Special:File...,8213152228518742662
1,tgm009985,sport,http://commons.wikimedia.org/wiki/Special:File...,4004049890319824358
2,tgm009985,sport,http://commons.wikimedia.org/wiki/Special:File...,10632294988152710640
3,tgm009985,sport,http://commons.wikimedia.org/wiki/Special:File...,52560277270845413
4,tgm009985,sport,http://commons.wikimedia.org/wiki/Special:File...,8618717754527682585
...,...,...,...,...
13819,tgm000686,award,http://commons.wikimedia.org/wiki/Special:File...,6543997110755914170
13820,tgm003584,work,http://commons.wikimedia.org/wiki/Special:File...,2158356187537040849
13821,tgm003584,work,http://commons.wikimedia.org/wiki/Special:File...,2470024329174724360
13822,tgm003584,work,http://commons.wikimedia.org/wiki/Special:File...,10366378009943306727


In [57]:
df.to_csv(path/'wikidata_urls.csv', index=False)

In [58]:
df.to_json(path/'wikidata_urls.json')

In [59]:
Path(path/'wikidata-images').mkdir(exist_ok=True)

In [60]:
save_path = Path(path/'wikidata-images')

In [61]:
df.columns

Index(['tgm', 'themeLabel', 'url', 'urlhash'], dtype='object')

In [70]:
def download_image(row, path=save_path):
    url = row.url
    hash = row.urlhash
    ext = url.split('.')[-1]
    fpath = f'{path}/{hash}.{ext}'
    if not Path(fpath).is_file():
        try:
            r = httpx.get(url, timeout=30)
            im = Image.open(io.BytesIO(r.content))
            im.save(fpath)
            return fpath         
        except Exception as e:
            logger.exception(e)
    else:
        pass

In [71]:
df['dl_fpath'] = df.progress_apply(download_image, axis=1)

In [72]:
df = df[df.dl_fpath.notna()].copy(deep=True)

In [73]:
df

,tgm,themeLabel,url,urlhash,dl_fpath


In [46]:
df = df.reset_index(inplace=True)

In [ ]:
df.to_csv(path/'wikidata_w_images.csv', index=False)

In [ ]:
df.to_json(path/'wikidata_w_images.json')